# WI-Projekt
Nr. 18

... <br>
... <br>
BA.WIN.22HS.VZa

... <br>
... <br>
BA.WIN.22HS.VZb

Pascal Kern <br>
kernpas1@students.zhaw.ch <br>
BA.WIN.22HS.VZa

## Setup

#### Get required Packages

In [43]:
import json
import os

from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

credentials = "credentials.json"

with open(credentials, 'r') as f:
    credentials = json.load(f)

# API-Schlüssel setzen
os.environ["OPENAI_API_KEY"] = credentials['openai']['api_key']
os.environ["GEMINI_API_KEY"] = credentials['gemini']['api_key']

In [ ]:
import boto3

bedrock_client = boto3.client(
    'bedrock-runtime',
    aws_access_key_id='your_access_key_id',
    aws_secret_access_key='your_secret_access_key',
    region_name='us-west-2'
)

#### Connect to DB

In [44]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'AntÃ´nio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

### Agent OpenAI

In [45]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) ## Which Large Language model are we using?
OpenAI_agent = create_sql_agent(llm, db=db, verbose=True) 


### Agent Google Gemini

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0.7, api_key=os.getenv("GEMINI_API_KEY")) ## mit temperatur 0.7 Gemini gives the same awnser like Openai when not using an agent type
Gemini_agent = create_sql_agent(llm, db=db, verbose=True) 

### Agent AWS Bedrock

In [47]:
from langchain_aws import BedrockLLM
import boto3

llm = BedrockLLM(model='eu.meta.llama3-2-1b-instruct-v1:0')
Llama_agent = create_sql_agent(llm, db=db, verbose=True)

### Execute

In [48]:
OpenAI_agent.invoke(
    "Welcher Artist hat am meisten Albums?"
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackI should query the schema of the Album and Artist tables to see which columns I can use to find the artist with the most albums.
Action: sql_db_schema
Action Input: Album, Artist
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/I can use the ArtistId column in the Album table to count the number of albums each artist has. I will write a query to group by ArtistId and cou

{'input': 'Welcher Artist hat am meisten Albums?', 'output': 'Iron Maiden'}

In [53]:
Gemini_agent.invoke(
    "Welcher Artist hat am meisten Albums?"
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe Artist table seems relevant to this question.  I should query the schema of the Artist table.
Action: sql_db_schema
Action Input: Artist
CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/Now I can query the database using the Artist table to find the artist with the most albums.
Action: sql_db_query
Action Input: SELECT Name, COUNT(AlbumId) AS AlbumCount FROM Artist JOIN Album ON Artist.ArtistId = Album.ArtistId GROUP BY Name ORDER BY AlbumCount DESC LIMIT 10[('Iron Maiden', 21), ('Led Zeppelin', 14), ('Deep Purple', 11), ('U2', 10), ('Metallica', 10), ('Ozzy Osbourne', 6), ('Pearl Jam', 5), ('Various Artists', 4), ('Van Halen', 4), ('Lost', 4)] I now know the final answer
Final Answer: Iron Maiden

> Finishe

{'input': 'Welcher Artist hat am meisten Albums?', 'output': 'Iron Maiden'}

In [54]:
Llama_agent.invoke(
    "Welcher Artist hat am meisten Albums?"
)

NameError: name 'Llama_agent' is not defined